<div id='firstRun'>
    <h2 style="color:red">If this is your first time 
    opening the script please click "Cell" then "Run All" in 
    the menu bar above.</h2>
</div>

# Auto Compare Modes v1.1
### Auto Compare Modes is a tool that bulk converts CORD Stat Act Comparison Reports into much more usefully fomatted Excel spreadsheets.

<b>Instructions:</b>
1. Download comparison reports as normal from CORD, and move the zip files to a folder that <b>contains only the comparisons reports you wish to analyse, and nothing else. Do not rename the zip files.</b>
2. Using the "Input Location" button, select the folder you just moved your reports to.
3. Using the "Output Location" button, select a location where you would like the resulting summary spreadsheet to be saved.
4. Click the "Run" button. A progress bar will appear underneath indicating how long is left before completion.
5. Once complete, 2 shortcut buttons will appear:
    - The "Open Output Folder" button will open a file explorer window at the output location you set.
    - The "Open Comparison Mode Summary" button will open the generated comparison using Excel.

<b>Limitations & Troubleshooting: </b>
* Currently, the script can only create one summary file for one comparison (eg INT21-
Bluebook). If other comparison zip's are found in the input file (eg DEV21-Bluebook), the comparison file will be skipped.
* Input and Output folders cannot be sharepoint locations and must either be local, or networked drive folders.
* If you experience any unexpected behaviour, first try resetting the script. To reset the script, in the menu bar above click "Kernel" then "Restart & Run All". <b> Note:</b> You do not have to reset the script after each run, only if you experience issues.

In [35]:
#Import modules for later use
import os, sys, glob, logging, time, threading, subprocess, traitlets
import numpy as np, pandas as pd, tkinter as tk
from datetime import datetime
from tkinter.filedialog import askdirectory
from zipfile import ZipFile
from ipywidgets import widgets
from IPython.display import display, HTML
from ipywidgets import widgets

In [36]:
class SelectFolderButton(widgets.Button):
    """A file widget that leverages tkinter.filedialog."""

    def __init__(self, desc):
        super(SelectFolderButton, self).__init__()
        # Create the button.
        self.description = desc
        self.icon = "square-o"
        self.style.button_color = "orange"
        self.layout = widgets.Layout(width='150px',height = '25px')
        # Set on click behavior.
        self.on_click(self.select_files)

    @staticmethod
    def select_files(b):
        """Generate instance of tkinter.filedialog.

        Parameters
        ----------
        b : obj:
            An instance of ipywidgets.widgets.Button 
        """
        # Create Tk root
        root = tk.Tk()
        # Hide the main window
        root.withdraw()
        # Raise the root to the top of all windows.
        root.call('wm', 'attributes', '.', '-topmost', True)
        # List of selected files will be set to b.value
        b.folder = askdirectory(title="Choose an INPUT folder")
        if not b.folder == '':
            b.icon = "check-square-o"
            b.style.button_color = "lightgreen"

In [37]:
#Global variable that detects whether a run is ongoing
running=False
#Function to open a folder via windows explorer
def openOutput(output_loc):
    """Uses the command prompt to open a windows explorer window at the location 'output_loc'.
    """
    fol = output_loc
    fol = fol.replace('/','\\')
    rp = os.getcwd()
    try:
        os.chdir(fol)
        os.chdir(rp)
        subprocess.call('explorer '+fol, shell=True)
    except:
        self.showMessage('Output folder does not exist!',mType='error',
                         detTxt='The following path is not an existing'+
                         ' directory:\n'+output_txt.text())
        os.chdir(rp)
def openSummary(loc):
    os.system('start "excel" "'+loc+'"')

In [38]:
#Create GUI
def run(b):
    global running
    if not running:
        running = True
        prog_bar.value = 0
        prog_bar.bar_style = ''
        run_button.description = 'Running...'
        run_button.disabled = True
        GUI_box_items = list(GUI_box.children)
        if prog_bar not in GUI_box_items:
            GUI_box_items.append(prog_bar)
        if open_out_button in GUI_box_items:
            GUI_box_items.remove(open_out_button)
        if open_button in GUI_box_items:
            GUI_box_items.remove(open_button)
        GUI_box.children=tuple(GUI_box_items)
        main(inp_lbl.value, out_lbl.value)
        """try:
            main(inp_lbl.value, out_lbl.value)
        except ValueError as ve:
            logger.critical('Main task failed to complete due to an error noted above.'
                            ' Please address errors then try again.')
            logger.debug(ve)
            fail()
        except Exception as e:
            logger.critical('Main task failed to complete due to an unknown error. '
                            'Please try resetting script using "Kernel"->"Restart & '
                            'Run All". If error persists contact Ross '
                            'at Ross.Gregory-Davies@ext.ons.gov.uk')
            logger.debug(e)
            critical_fail()"""
def fail():
    global running
    running = False
    run_button.disabled = False
    pThread.stop()
    run_button.description = 'Run'
    prog_bar.bar_style = 'danger'
def critical_fail():
    global running
    running = False
    run_button.button_style = 'danger'
    pThread.stop()
    prog_bar.bar_style = 'danger'
    run_button.disabled = True
    run_button.description = 'Failed. Reset Required.'
def enable_run(b):
    if inp_button.icon == "check-square-o" and out_button.icon == "check-square-o":
        #GUI_box.children += (run_button,)
        run_button.button_style = "success"
        run_button.disabled = False
def update_button(b):
    if inp_button.icon == "check-square-o":
        inp_lbl.value = inp_button.folder
    if out_button.icon == "check-square-o":
        out_lbl.value = out_button.folder
    enable_run(b)
def completed_run(out_loc):
    global running
    running = False
    run_button.disabled = False
    run_button.description = 'Run'
    prog_bar.bar_style = 'success'
    open_out_button.on_click(lambda b: openOutput(out_lbl.value)) 
    open_button.on_click(lambda b: openSummary(out_loc))
    GUI_box.children=tuple(list(GUI_box.children)+[open_out_button, open_button])
lbl_layout = widgets.Layout(width='auto', height='', max_width='1920px')
inp_button = SelectFolderButton("Input Location")
inp_lbl = widgets.Label('Please select...', layout=lbl_layout)
out_button = SelectFolderButton("Output Location")
out_lbl = widgets.Label('Please select...', layout=lbl_layout)
prog_bar = widgets.FloatProgress(
                value=0,
                min=0,
                max=100,
                description='Progress:',
                bar_style='', # 'success', 'info', 'warning', 'danger' or ''
                orientation='horizontal',
                layout=widgets.Layout(width='auto',height = '')
            )
inp_button.on_click(update_button)
out_button.on_click(update_button)
folder_layout = widgets.Layout(width='auto', height='')
inp_both = widgets.HBox(children=[inp_button, inp_lbl])
out_both = widgets.HBox(children=[out_button, out_lbl])
folder_box = widgets.VBox(children=[inp_both, out_both], layout=folder_layout)
run_button = widgets.Button(description='Run', disabled=True,
                                  layout=widgets.Layout(width='auto',height = '50px'))
open_out_button = widgets.Button(description='Open Output Folder', button_style = 'info',
                                  layout=widgets.Layout(width='auto',height = '25px'))
open_button = widgets.Button(description='Open Compare Mode Summary', button_style = 'info',
                                  layout=widgets.Layout(width='auto',height = '25px'))
run_button.on_click(run)
GUI_box = widgets.VBox(children=[folder_box, run_button])
GUI_box

In [39]:
#Class for managing the progress bar in a seperate thread
class progThread(threading.Thread):
    """Creates a new thread to update the progress bar smoothly without causing delays to the
    existing process.
    Usage: progThread(start value, amount to increase, delay between increase, progress bar object)
    Example: progThread(10, 60, 2, prog_bar)
             This would create a thread to update prog_bar with values starting at 10 and
             ending at 70, using 1 step increments every 2 seconds.
    """
    def __init__(self, progStart, amount, delay, prog):
        threading.Thread.__init__(self)
        self._stop_event = threading.Event()
        self.progStart = progStart
        self.amount = amount
        self.delay = delay
        self.prog = prog
        self.kill = False
    def run(self):
        i = self.progStart
        lastCall = time.time()
        while i<self.progStart+self.amount:
            curTime = time.time()
            if self.kill:
                break
            if curTime-lastCall >= self.delay:
                lastCall = curTime
                self.prog.value = i
                i+=1
        if not self.stopped():
            self.stop()
    def stop(self):
        self.kill = True
        self._stop_event.set()
    def stopped(self):
        return self._stop_event.is_set()

In [40]:
#Custom logging handler that ignores debug and info calls
class GUIHandler(logging.StreamHandler):
    """Custom stream handler to allow writing all levels to file but only
    non-info/debug levels to the users console.
    """
    def __init__(self):
        logging.StreamHandler.__init__(self)
    def emit(self, record):
        try:
            msg = self.format(record)
            if not '[DEBUG]' in msg and not '[INFO]' in msg:
                self.stream.write(msg)
                self.stream.write(self.terminator)
            self.flush()
        except (KeyboardInterrupt, SystemExit):
            raise
        except:
            self.handleError(record)

In [41]:
def setupFilepaths():
    """Sets up the INPUT and OUTPUT filepaths by asking the user to select the
    folders.
    Returns a list of filepath strings based on input arguments.
    """
    root = tk.Tk()
    filepaths = []
    inpFol = askdirectory(title="Choose an INPUT folder")
    filepaths.append(inpFol)
    outFol = askdirectory(title="Choose an OUTPUT folder")
    filepaths.append(outFol)
    root.destroy()
    if inpFol == '':
        logger.error('An INPUT folder must be selected.')
        raise ValueError('INPUT Folder not selected.')
    if outFol == '':
        logger.error('An OUTPUT folder must be selected.')
        raise ValueError('OUTPUT Folder not selected.')
    return filepaths

In [42]:
def unzipFiles(file):
    """Unzips all zip files within the specified folder.
    """
    fileTitle = os.path.splitext(file)[0]
    logger.info('Unzipping '+fileTitle+'...')
    with ZipFile(file, 'r') as zipObj:
        zipObj.extractall(fileTitle)    

In [43]:
def compileCO(summaryDf, idxName):
    """Compiles summary dataframe's into coDf, where each summaryDf is a row.
    """
    global coDf
    for i, column in enumerate(summaryDf.loc[0,:]):
        coDf.loc[idxName, column] = summaryDf.loc[1, i]
    coDf = coDf.replace('N', np.nan)

In [44]:
def readSummmary(dirname):
    """Reads the Comparison Report file within specified directory (dirname).
    Returns the summary file dataframe, name of the system, the source and 
    target modes.
    
    Skips first 5 rows of the file and transposes the datframe to create
    summaryDf. headerDf is also created which is only used the read the file
    header which contains the source and target of the comparison for later
    checks.
    """
    global supportedTypes
    files = []
    for (dirpath, dirnames, filenames) in os.walk(dirname):
        files.extend(filenames)
        break
    for file in files:
        file = dirpath+'/'+file
        if '_Comparison_Report_Summary_' in file:
            splitDir = dirname.split('\\')[-1]\
                            .split('_Stat_Act_Comparision_Report_')
            summaryDf = pd.read_csv(file, skiprows=5, header=None,
                                       encoding='unicode_escape')
            headerDf = pd.read_csv(file, header=None, sep='|',
                                       encoding='unicode_escape')
            infoLine = headerDf.loc[2, 0]
            splitInfo = infoLine.split('/')
            src = splitInfo[1].split(' ')[0]
            trg = splitInfo[2]
            summaryDf = summaryDf.T
        #No_objects_found.txt fills report summary files when a Stat Act Comp
        #is run on an empty Statistical Activity.
        if 'No_objects_found' in file:
            summaryDf = pd.DataFrame()
            src = 'invalid'
            trg = 'invalid'
            splitDir =  dirname.split('\\')[-1]\
                            .split('_Stat_Act_Comparision_Report_')
    for i, col in enumerate(summaryDf.iloc[0]):
        if col not in list(supportedTypes):
            summaryDf.drop(i, axis=1, inplace=True)
            logger.warning('Unable to read "'+col+'" in "'+splitDir[0]+'" as'
                           ' it is currently unsupported.\nThese values will'
                           ' have to be checked manually.')
    summaryDf.columns = range(summaryDf.shape[1])
    return summaryDf, splitDir[0], src, trg

In [45]:
def formatComparison(writer):
    """Formats the Comparison Overview worksheet.
    
    Sets column widths and assigns relevant formatting to cells.
    """
    global coDf
    wb = writer.book
    ws = writer.sheets['Comparison Overview']
    r = len(coDf.iloc[:, 0]) + 1
    diff_format = wb.add_format({'bg_color': '#f67c7c',
                                 'border_color': '#d60000',
                                 'border': 5,
                                 'align': 'center',
                                 'valign': 'vcenter',
                                 'bold': True,
                                 'font_color': 'white'})
    empty_format = wb.add_format({'bg_color': 'white',
                                 'border': 1,
                                 'align': 'center',
                                 'valign': 'vcenter'})
    diffDf = coDf.replace(np.nan, '')
    for c in range(len(coDf.iloc[0, :])):
        for r in range(len(coDf.iloc[:, 0])):
            if diffDf.iloc[r, c] != '':
                ws.write(r+1, c+1, 'Different', diff_format)
            else:
                ws.write(r+1, c+1, '', empty_format)
    ws.set_column(0, 0, 30)
    ws.set_column(1, 1, 14)
    ws.set_column(2, 2, 10)
    ws.set_column(3, 3, 10)
    ws.set_column(4, 4, 11)
    ws.set_column(5, 5, 13)
    ws.set_column(6, 6, 12)
    ws.set_column(7, 7, 12)
    ws.set_column(8, 8, 10)
    ws.set_column(9, 9, 16)
    ws.set_column(10, 10, 15)
    ws.set_column(11, 11, 12)

In [46]:
def adjustCO(sys, field):
    """Adjusts the Comparison Overview sheet in the case a badly encoded csv
    is found.
    """
    global writer
    global coDf
    wb = writer.book
    ws = writer.sheets['Comparison Overview']
    col = None
    for c, title in enumerate(coDf.columns):
        if title == field:
            col = c
    if col == None:
        logger.error('Field not found!')
    row = None
    for r, idx in enumerate(coDf.index):
        if idx == sys:
            row = r
    if row == None:
        logger.error('System not found!')
    ws.write(row+1, col+1, 'Review', wb.add_format({'border': 5,
                                                  'bg_color': 'red',
                                                  'border_color': '#cc3300',
                                                  'align': 'center',
                                                  'valign': 'vcenter',
                                                  'bold': True,
                                                  'font_color': 'white'}))

In [47]:
def createSheetDf(sheet):
    """Returns the dataframe holding the comparison field data for all systems
    where the comparison report showed differences.
    """
    global curDir
    files = []
    #Predefine the sheet dictionary which helps read the csv files
    sheetDict = dict([('CLASSIFICATION', 'Classifications'),
                     ('TASK', 'Tasks'),
                     ('PARAMETERS', 'Parameters'),
                     ('MAPPINGS', 'Classification_Mappings'),
                     ('CALCULATIONS', 'Calculations'),
                     ('CON CHECKS', 'Consistency_Checks'),
                     ('IMPORT DEFS', 'Import_Definitions'),
                     ('DATASETS', 'Dataset_Definitions'),
                     ('SEAS EXPORT DEFS', 'Seas_Analysis_Export_Definitions'),
                     ('VISUALISATIONS', 'Visualisations'),
                     ('EXPORT DEFS', 'Export_Definitions')])
    #Convert column header (sheet) to filename (sheetToFile) based on sheetDict
    try: 
        sheetFileName = sheetDict[sheet]
    except:
        logger.error('Unrecognised sheet "' + sheet + '", skipping...')
        return
    #Create a list containing all files
    for (dirpath, dirnames, filenames) in os.walk(curDir):
        files.extend(filenames)
        break
    #loop through files until we find the one we're looking for
    for file in files:
        file = dirpath+'/'+file
        if sheetFileName in file:
            dataStart = None
            basAtr = None
            #Try to read the csv, if the encoding is unrecognised the send
            #it to adjustCO to show it's been skipped on output sheet.
            try:
                prelimDf = pd.read_csv(file, header=None, sep='\n',
                               encoding = 'unicode_escape')
            except:
                logger.error('BAD ENCODING IN '+curDir+' \nSKIPPING DATA,'+
                      'THIS WILL NEED TO BE REVIEWED MANUALLY...')
                adjustCO(curDir.split('_')[0], sheet)
                continue
            for i, line in enumerate(prelimDf.iloc[:,0]):
                #Find where the data starts
                if 'Name,Match' in line:
                    dataStart = i+2
                    #Added placeholder to account for final commas
                    headerNames = line.split(',')+['Placeholder']
                #Grab the basic attribute data as it's variable between files
                if 'Basic Attributes: ' in line:
                    basAtr = line.split('Basic Attributes: ')[-1]
                    basAtr = basAtr[:-2]
            #Get the sheet dataframe containing it's predefined columns
            sheetDf = pd.read_csv(file, skiprows=dataStart, quotechar='"',
                               names=headerNames, encoding = 'unicode_escape')
            sheetDf.drop('Placeholder', inplace=True, axis=1)
            #Create a dataframe to hold data from all files
            finalDf = pd.DataFrame(columns=['System',
                                            'Name',
                                            'Location',
                                            'Differences'])
            #Loop through all rows of sheetDf and append the rows that showed
            #differences to the finalDf, listing their differences in the
            #Differences column
            for r, item in enumerate(sheetDf.loc[:, 'Identical']):
                rowDf = pd.DataFrame()
                if item != 'Y':
                    diffStr = ''
                    rowDf.loc[0, 'System'] = curDir.split('\\')[-1]\
                                                .split('_')[0]
                    if  not pd.notnull(item) and not\
                        pd.notnull(sheetDf.loc[r,'Match']):
                        rowDf.loc[0, 'Name'] = sheetDf.loc[r, 'Items']
                        if sheetDf.loc[r, 'Groups']=='Descriptions different':
                            diffStr = sheetDf.loc[r, 'Groups']
                            rowDf.loc[0, 'Location'] = 'In Both'
                        else:
                            rowDf.loc[0, 'Location'] = sheetDf.loc[r, 'Groups']
                    else:
                        rowDf.loc[0, 'Name'] = sheetDf.loc[r, 'Name']
                        rowDf.loc[0, 'Location'] = sheetDf.loc[r, 'Match']
                    diffs = []
                    if item == 'N':
                        for i, x in enumerate(sheetDf.iloc[r, 3:]):
                            if pd.notnull(x):
                                colHead = sheetDf.columns[i+3]
                                if colHead == 'Basic Attributes':
                                    diffs.append('Basic Attributes (' + 
                                                 basAtr + ')')
                                else:
                                    diffs.append(colHead)
                    if diffs != []:
                        diffStr = (', ').join(diffs)
                    rowDf.loc[0, 'Differences'] = diffStr
                    #Final check to make sure data isn't just misplaced headers
                    if rowDf.loc[0, 'Name'] == 'Name' and \
                    rowDf.loc[0, 'Location'] == 'Match':
                        continue
                    finalDf = finalDf.append(rowDf, ignore_index=True)
                    
            return finalDf

In [48]:
def createSheet(sheet):
    """Creates and formats the sheet for a specific comparison field (sheet).
    """
    global writer
    global dirs
    global coDf
    global curDir
   
    changed = []
    dfList = []
    logger.info('Creating sheet for ' + sheet + '...')
    #Loop through all systems in comparison overview dataframe to find those
    #with differences
    for i, item in enumerate(coDf.loc[:, sheet]):
        #If system has changed for this comparison field append it to the list
        if item == 'Y':
            changed.append(coDf.index[i])
    #Create a list of dataframes for each system that has changes in the
    #comparison field (sheet)
    for changedSys in changed:
        for aDir in dirs:
            if changedSys in aDir:
                curDir = aDir
                dfList.append(createSheetDf(sheet))
    bigDf = pd.DataFrame()
    #Compile the list of dataframes into one big dataframe
    for df in dfList:
        df = df.replace('',np.nan)
        df.dropna(how='all', subset=['Name','Location','Differences'],
                  inplace=True)
        df = df.replace(np.nan, '')
        bigDf = bigDf.append(df)
    #Create a worksheet using xlsxwriter
    wb = writer.book
    ws = wb.add_worksheet(sheet)
    #Create variable to hold the system name, as when a system changes we
    #want a larger border to visually seperate systems
    prevSys = None
    #Loop through each row in the bigDf and each column item to the sheet in
    #the proper formatting
    for r, system in enumerate(bigDf.loc[:, 'System']):
        std_format = wb.add_format({'bg_color': 'white',
                                    'border': 1,
                                    'align': 'left',
                                    'valign': 'vcenter'})
        sys_format = wb.add_format({'bg_color': 'white',
                                    'border': 1,
                                    'right': 5,
                                    'left': 5,
                                    'align': 'center',
                                    'bold': True,
                                    'valign': 'vcenter'})
        if r == 0:
            ws.write(0,0, 'System', wb.add_format({'border': 5,
                                                   'bold': True,
                                                   'align': 'center',
                                                   'valign': 'vcenter'}))
            ws.write(0,1, 'Name', wb.add_format({'top': 5,
                                                 'bottom': 5,
                                                 'left': 5,
                                                 'right': 1,
                                                 'bold': True,
                                                 'align': 'center',
                                                 'valign': 'vcenter'}))
            ws.write(0,2, 'Location', wb.add_format({'top': 5,
                                                 'bottom': 5,
                                                 'left': 1,
                                                 'right': 1,
                                                 'bold': True,
                                                 'align': 'center',
                                                 'valign': 'vcenter'}))
            ws.write(0,3, 'Differences', wb.add_format({'top': 5,
                                                 'bottom': 5,
                                                 'left': 1,
                                                 'right': 5,
                                                 'bold': True,
                                                 'align': 'center',
                                                 'valign': 'vcenter'}))
        #Sets up the thick borders for system separation
        if system != prevSys:
            std_format.set_top(5)
            sys_format.set_top(5)
            prevSys = system
        
        #Sets a thick border for the final line of the dataset
        if r == len(bigDf.iloc[:, 0])-1:
            std_format.set_bottom(5)
            sys_format.set_bottom(5)
        
        #Writes the data to the sheet for each item in each column
        for c, info in enumerate(bigDf.iloc[r, :]):
            if c == 0:
                form = sys_format
            else:
                form = std_format
            ws.write(r+1, c, info, form)
        #Get the cell range of the dataset as a string
        cellStr = 'E2:E' + str(len(bigDf.iloc[:,0])+1)
        #Make the far right border of the dataset thick
        ws.conditional_format(cellStr, {'type': 'blanks',
                                   'format': wb.add_format({'left': 5})
                                   })
                                        
    #Set the column widths
    ws.set_column(0, 0, 27)
    ws.set_column(1, 1, 40)
    ws.set_column(2, 2, 11)
    ws.set_column(3, 3, 126)

In [49]:
#Runs the main function if the script itself is being run and not being imported
def main(inpFol, outFol):
    """Runs the overall task, converting zip files in the INPUT folder into
    an xlsx sheet in the OUTPUT folder.
    """
    global coDf, writer, dirs, logger, running, progBar, pThread
    coDf = pd.DataFrame(columns = ['CLASSIFICATION',
                               'TASK',
                               'CLASSIFICATION MAPPING',
                               'PARAMETER',
                               'CALCULATION DEFINITION',
                               'CONSISTENCY CHECK DEFINITION',
                               'TIMESERIES IMPORT DEFINITION',
                               'DATASET DEFINITION',
                               'SEAS EXPORT DEFINITION',
                               'VISUALISATION DEFINITION',
                               'TIMESERIES EXPORT DEFINITION'])
    #Set up the logger
    log = GUIHandler()
    fileLog = logging.FileHandler(filename='AutoCompareModes.log', mode='w')
    logging.getLogger("AutoCompareModes").handlers = []
    log.setFormatter(logging.Formatter(
        "[%(levelname)s] %(message)s"))
    fileLog.setFormatter(logging.Formatter(
        "[%(asctime)s] ->%(lineno)d [%(levelname)s] %(message)s"))
    logging.getLogger("AutoCompareModes").addHandler(log)
    logging.getLogger("AutoCompareModes").addHandler(fileLog)
    logging.getLogger("AutoCompareModes").setLevel(logging.DEBUG)
    logger = logging.getLogger("AutoCompareModes")
    logger.propagate = False
    #Start the progress bar thread
    pThread = progThread(0,30,0.1,prog_bar)
    pThread.start()
    #Reset the source and target modes
    origSrc = None
    origTrg = None
    dirs = []
    #Valid = True when a valid comparison report file is found in the INPUT folder 
    valid = False
    #Unzip all zip files
    if list(glob.glob(inpFol+'/'+'*.zip')) == []:
        logger.error('NO INPUT FILES FOUND. Place CORD Stat Act Comparison zip '+
              'files in the INPUT file and re-run.')
        raise ValueError('No input files found in INPUT folder.')
    for filepath in glob.glob(inpFol+'/'+'*.zip'):
        file = filepath.split('\\')[-1]
        if 'Stat_Act_Comparision_Report' not in file:
            logger.warning(file + ' is not a Comparison Report, this file will be '+
                  'skipped.')
            continue
        if not valid:
            valid = True
        skip = False
        for stat in dirs:
            cur = file.split('_Stat_Act_Comparision_Report_')[0]
            if stat.split('_Stat_Act_Comparision_Report_')[0] == cur:
                logger.warning('There are multiple Reports for the Stat Act: ' + cur +
                      '. Only the file "' + stat + '" will be analysed.')
                skip=True
        if skip: continue
        unzipFiles(filepath)
        dirs.append(os.path.splitext(filepath)[0])
    delay = 0.1*len(dirs)
    pThread.stop()
    pThread = progThread(30,20,delay,prog_bar)
    pThread.start()
    if not valid:
        logger.error('No comparison reports were found in the INPUT folder!')
        raise ValueError('No comparison reports in INPUT folder.')
    #Read all directories to create the comparison dataset
    for aDir in dirs:
        logger.info('Reading summary for '+aDir.split('\\')[-1]+'...')
        sList, idxName, src, trg = readSummmary(aDir)
        #If the zip file contains no files it will return 'invalid'
        #If invalid is found, throw and error and skip the folder
        if 'invalid' in {src, trg}:
            logger.error('This report contains no information! Skipping...')
            coDf = coDf.append(pd.Series(name=idxName))
            continue
        if(origSrc == None and origTrg == None):
            origSrc = src
            origTrg = trg
        if src == origSrc and trg == origTrg:
            logger.info('Adding to comparison...')
            compileCO(sList, idxName)
        else:
            logger.error('Source and target were different!' + 
                  ' Skipping comparison...')
    pThread.stop()
    pThread = progThread(50,49,0.5,prog_bar)
    pThread.start()
    logger.info('Creating Comparison Overview...')
    date = datetime.today().strftime('%d-%m-%Y')
    compTitle = 'Comparison Overview (' + src + '-' + trg + ') ' + date +\
    '.xlsx'
    sameFiles = []
    for file in glob.glob(outFol+'/'+'*.xlsx'):
        if compTitle[:-5] in file:
            sameFiles.append(file)
    if os.path.exists(outFol+'/'+compTitle):
        compTitle = compTitle[:-5] + '(' + str(len(sameFiles)) + ').xlsx'
    writer = pd.ExcelWriter(outFol+'/'+compTitle, engine='xlsxwriter')
    coDf.columns = ['CLASSIFICATION',
                    'TASK',
                    'MAPPINGS',
                    'PARAMETERS',
                    'CALCULATIONS',
                    'CON CHECKS',
                    'IMPORT DEFS',
                    'DATASETS',
                    'SEAS EXPORT DEFS',
                    'VISUALISATIONS',
                    'EXPORT DEFS']
    coDf.to_excel(writer, sheet_name='Comparison Overview')
    logger.info('Applying formatting to Comparison Overview...')
    formatComparison(writer)

    #Create a comparison sheet for every comparison field that has a changed
    #system
    coDfAdj = coDf.dropna(how='all', axis=1)
    for col in coDfAdj.columns:
        createSheet(col)
    #Save the file
    writer.save()
    pThread.stop()
    prog_bar.value = 100
    logger.info('Done!')
    completed_run(outFol+'/'+compTitle)

In [50]:
supportedTypes = ['CLASSIFICATION',
                  'TASK',
                  'CLASSIFICATION MAPPING',
                  'PARAMETER',
                  'CALCULATION DEFINITION',
                  'CONSISTENCY CHECK DEFINITION',
                  'TIMESERIES IMPORT DEFINITION',
                  'DATASET DEFINITION',
                  'SEAS EXPORT DEFINITION',
                  'VISUALISATION DEFINITION',
                  'TIMESERIES EXPORT DEFINITION']

In [51]:
HTML('''<script>
code_show=true;
var x = document.getElementById("firstRun");
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 x.style.display = "none";
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this notebook is hidden by default for easier reading.
To toggle the raw code on/off, click <a href="javascript:code_toggle()">here</a>.''')